# Büyük Veri Final

## Kaggle Üzerinde Proje Geliştirme

Merve TAFRALI - 160202100

Bu projedeki amacım veri setinde olan verilerden yararlanarak bir sonraki ayın satış tahminlerini gerçekleştirmektir. Öncelikle versetini tanıyabilmek adına veri görselleştirme adımlarını kullanacağım.

In [ ]:
from math import ceil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.palplot(sns.husl_palette(10))
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler

from math import ceil
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop

%matplotlib inline

#dataları okuma

train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
sample = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_cats = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')

### Veri Görselleştirme

In [ ]:
def bilgi(data):
    print("-----İlk 5 Kayıt-----")
    print(data.head(5))
    print("-----Bilgiler-----")
    print(data.info())
    print("-----Data Türleri-----")
    print(data.dtypes)
    print("-----Eksik Değerler-----")
    print(data.isnull().sum())
    print("-----Null Değerler-----")
    print(data.isna().sum())
    

In [ ]:
bilgi(train)


In [ ]:
bilgi(test)

In [ ]:
bilgi(sample)

In [ ]:
bilgi(items)

In [ ]:
bilgi(item_cats)

In [ ]:
bilgi(shops)

In [ ]:
#outliers
train = train[train.item_price <= 100000]
train = train[train.item_cnt_day <= 1000]


median = train[(train.shop_id == 32) & (train.item_id == 2973) & (train.date_block_num == 4) & (train.item_price > 0)].item_price.median()
train.loc[train.item_price < 0, 'item_price'] = median

test_shops = test.shop_id.unique()
train = train[train.shop_id.isin(test_shops)]
test_items = test.item_id.unique()
train = train[train.item_id.isin(test_items)]
train.head()

İkiliyen dataların çıakrılması için basit bir fonksiyon

In [ ]:

def drop_duplicate(data, subset):
    print('Before drop shape:', data.shape)
    before = data.shape[0]
    data.drop_duplicates(subset,keep='first', inplace=True) 
    data.reset_index(drop=True, inplace=True)
    print('After drop shape:', data.shape)
    after = data.shape[0]
    print('Total Duplicate:', before-after)

In [ ]:
subset = ['date', 'date_block_num', 'shop_id', 'item_id','item_cnt_day']
drop_duplicate(train, subset = subset)

In [ ]:
sns.scatterplot(train['item_cnt_day'],train['item_price'])

In [ ]:
item_cats.head()

Kategoriler rusça olduğundan daha kolay ve daha rahat görselleştirmek için uygulanan adımlar,

In [ ]:
def r(x):
    if 'PC' in x:
        return 'PC'
    elif 'Live!' in x:
        return 'Live!'
    elif 'CD' in x:
        return 'CD'
    elif 'PS2' in x:
        return 'PS2'
    elif 'PSVita' in x:
        return 'PSVita'
    elif 'Windows' in x:
        return 'Windows'
    elif 'DVD' in x:
        return 'DVD'
    elif 'MP3' in x:
        return 'MP3'
    elif 'PS3' in x:
        return 'PS3'
    elif 'PSP' in x:
        return 'PSP'
    elif 'PS4' in x:
        return 'PS4'
    elif 'PSVita' in x:
        return 'PSVita'
    elif 'XBOX 360' in x:
        return 'XBOX 360'
    elif 'XBOX ONE' in x:
        return 'XBOX ONE'
    elif 'Blu-Ray 3D' in x:
        return 'Blu-Ray 3D'
    elif 'Blu-Ray 4K' in x:
        return 'Blu-Ray 4K'
    
    else:
        return 'Others'
    

    
item_cats['item_category_name']=item_cats['item_category_name'].apply(r)

In [ ]:
plt.figure(figsize=(15,10))
sns.countplot(item_cats['item_category_name'])

Amacımız aylık satış tahmini olduğundan dolayı her ay ki satışları görmek için bir görselleştirme adımı gerçekleştirilmelidir.

In [ ]:

ax = sns.distplot(train.groupby('date_block_num').sum()['item_cnt_day'], color="r")
plt.show()




Date tipini değiştirmemiz gerekiyor, yukarıda yaptığımız analizleri tekrar uyguluyorum

In [ ]:
train2=pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv",index_col='date',parse_dates=['date'])
train2 = train2[train2.shop_id.isin(test_shops)]
test_items = test.item_id.unique()
train2 = train2[train2.item_id.isin(test_items)]
train2 = train2[train2.item_price <= 100000]
train2 = train2[train2.item_cnt_day <= 1000]
median = train2[(train2.shop_id == 32) & (train2.item_id == 2973) & (train2.date_block_num == 4) & (train2.item_price > 0)].item_price.median()
train2.loc[train2.item_price < 0, 'item_price'] = median


train2["item_price"][:'2014-01-01'].plot(figsize=(16,10),legend=True,color='r')
train2["item_price"]['2014-01-01':'2015-01-01'].plot(figsize=(16,10),legend=True,color='b')
train2["item_price"]['2015-01-01':].plot(figsize=(16,10),legend=True,color='g')
plt.xlabel("Dates")
plt.ylabel("Item price rise")
plt.title("Item pices vs Date")

# Makine Öğrenmesinin Uygulanması

Üç tür makine öğrenimi vardır

1. Denetimli Makine Öğrenimi
2. Denetimsiz Makine Öğrenimi
3. Takviye Makinesi Öğrenimi

**Denetimli Makine Öğrenimi**

Hem girdi hem de istenen çıktı verilerinin sağlandığı bir öğrenme türüdür. Girdi ve çıktı verileri, gelecekteki veri işleme için bir öğrenme temeli sağlamak üzere sınıflandırma için etiketlenir. Bu algoritma, belirli bir yordayıcı kümesinden (bağımsız değişkenler) tahmin edilmesi gereken bir hedef / sonuç değişkeninden (veya bağımlı değişken) oluşur. Bu değişkenler setini kullanarak, girişleri istenen çıkışlarla eşleştiren bir fonksiyon üretiyoruz. Eğitim süreci, model eğitim verileri üzerinde istenen bir doğruluk seviyesine ulaşana kadar devam eder.

**Denetimsiz Makine Öğrenimi**

Gözetimsiz öğrenme, ne sınıflandırılmamış ne de etiketlenmemiş bilgileri kullanan ve algoritmanın rehberlik olmadan bu bilgiler üzerinde hareket etmesine izin veren bir eğitimdir. Denetimsiz öğrenmenin arkasındaki ana fikir, makineleri büyük miktarlarda çeşitli verilere maruz bırakmak ve öğrenmesine izin vermektir. ve verilerden çıkar. Ancak, makineler öncelikle verilerden öğrenilecek şekilde programlanmalıdır.

Gözetimsiz öğrenme sorunları ayrıca kümeleme ve ilişkilendirme sorunları olarak gruplandırılabilir.

**Takviye Makinesi Öğrenimi - Semi-Supervised**

Verisetinin bir kısmını denetimle bir kısmını da denetimsiz öğrenme yardımıyla eğiterek gerçekleştirilir.


In [ ]:
from catboost import CatBoostRegressor
from catboost import Pool
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import gc
import itertools
import xgboost
from xgboost import plot_importance
from xgboost import XGBRegressor

from lightgbm import LGBMRegressor

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


Bu fonksiyon https://www.kaggle.com/gemartin/load-data-reduce-memory-usage adresinde vardır

In [ ]:
#https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
  
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col]#.astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
shops = reduce_mem_usage(shops)

In [ ]:
shops.head()

Veri setinde düzenlemer

In [ ]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]

item_cats['split'] = item_cats['item_category_name'].str.split('-')
item_cats['type'] = item_cats['split'].map(lambda x: x[0].strip())
item_cats['type_code'] = LabelEncoder().fit_transform(item_cats['type'])

item_cats['subtype'] = item_cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
item_cats['subtype_code'] = LabelEncoder().fit_transform(item_cats['subtype'])
item_cats = item_cats[['item_category_id','type_code', 'subtype_code']]

items.drop(['item_name'], axis=1, inplace=True)

In [ ]:
%%time
full_data = []
for date_block_num in range(34):
    all_shop_id = train2[train2.date_block_num==date_block_num].shop_id.unique()
    all_item_id = train2[train2.date_block_num==date_block_num].item_id.unique()
    full_data.append(np.array(list(itertools.product([date_block_num], all_shop_id, all_item_id))))

full_data = np.vstack(full_data)
full_data = pd.DataFrame(full_data, columns=["date_block_num", "shop_id", "item_id"])
full_data.fillna(0, inplace=True)
full_data.sort_values(["date_block_num", "shop_id", "item_id"])

In [ ]:
train['revenue'] = train['item_price'] *  train['item_cnt_day']

In [ ]:
full_data = reduce_mem_usage(full_data)

In [ ]:
group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

full_data = pd.merge(full_data, group, on=['date_block_num','shop_id','item_id'], how='left')
full_data['item_cnt_month'] = (full_data['item_cnt_month']
                                .fillna(0)
                                .clip(0,20)
                                .astype(np.float16))

In [ ]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

In [ ]:
full_data = pd.concat([full_data, test], ignore_index=True, sort=False, keys=['date_block_num','shop_id','item_id'])
full_data.fillna(0, inplace=True)

In [ ]:
full_data.columns, item_cats.columns

In [ ]:
full_data = pd.merge(full_data, shops, on=['shop_id'], how='left')
full_data = pd.merge(full_data, items, on=['item_id'], how='left')
full_data = pd.merge(full_data, item_cats, on=['item_category_id'], how='left')
full_data['city_code'] = full_data['city_code'].astype(np.int8)
full_data['item_category_id'] = full_data['item_category_id'].astype(np.int8)
full_data['type_code'] = full_data['type_code'].astype(np.int8)
full_data['subtype_code'] = full_data['subtype_code'].astype(np.int8)

In [ ]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [ ]:
%%time
full_data = lag_feature(full_data, [1,2,3,6,12], 'item_cnt_month')

In [ ]:
%%time
date_item_group = full_data.groupby(["date_block_num", "item_id"]).agg({"item_cnt_month": ["mean"]})
date_item_group.columns = ["date_item_avg_item_cnt"]
date_item_group.reset_index(inplace=True)
full_data = pd.merge(full_data, date_item_group, on=["date_block_num", "item_id"], how="left")
full_data = lag_feature(full_data, [1, 2, 3, 6, 12], "date_item_avg_item_cnt")
full_data.drop(["date_item_avg_item_cnt"], axis=1, inplace=True)

date_item_group = full_data.groupby(["date_block_num", "shop_id"]).agg({"item_cnt_month": ["mean"]})
date_item_group.columns = ["date_shop_avg_item_cnt"]
date_item_group.reset_index(inplace=True)
full_data = pd.merge(full_data, date_item_group, on=["date_block_num", "shop_id"], how="left")
full_data = lag_feature(full_data, [1, 2, 3, 6, 12], "date_shop_avg_item_cnt")
full_data.drop(["date_shop_avg_item_cnt"], axis=1, inplace=True)

date_item_group = full_data.groupby(["date_block_num", "item_id", "shop_id"]).agg({"item_cnt_month": ["mean"]})
date_item_group.columns = ["date_item_shop_avg_item_cnt"]
date_item_group.reset_index(inplace=True)
full_data = pd.merge(full_data, date_item_group, on=["date_block_num", "item_id", "shop_id"], how="left")
full_data = lag_feature(full_data, [1, 2, 3], "date_item_shop_avg_item_cnt")
full_data.drop(["date_item_shop_avg_item_cnt"], axis=1, inplace=True)

date_shop_cat_group = full_data.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_month': ['mean']})
date_shop_cat_group.columns = ['date_shop_cat_avg_item_cnt']
date_shop_cat_group.reset_index(inplace=True)
full_data = pd.merge(full_data, date_shop_cat_group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
full_data = lag_feature(full_data, [1], 'date_shop_cat_avg_item_cnt')
full_data.drop(['date_shop_cat_avg_item_cnt'], axis=1, inplace=True)

item_group = train.groupby(["item_id"]).agg({"item_price": ["mean"]})
item_group.columns = ["item_avg_item_price"]
item_group.reset_index(inplace=True)
full_data = pd.merge(full_data, item_group, on=["item_id"], how="left")

date_item_group = train.groupby(["date_block_num", "item_id"]).agg({"item_price": ["mean"]})
date_item_group.columns = ["date_item_avg_item_price"]
date_item_group.reset_index(inplace=True)
full_data = pd.merge(full_data, date_item_group, on=["date_block_num", "item_id"], how="left")

lags = [1,2,3,4,5,6]
full_data = lag_feature(full_data, lags, 'date_item_avg_item_price')

for i in lags:
    full_data['delta_price_lag_'+str(i)] = \
        (full_data['date_item_avg_item_price_lag_'+str(i)] - full_data['item_avg_item_price']) / full_data['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
full_data['delta_price_lag'] = full_data.apply(select_trend, axis=1)
full_data['delta_price_lag'] = full_data['delta_price_lag'].astype(np.float16)
full_data['delta_price_lag'].fillna(0, inplace=True)

In [ ]:
group = train.groupby(['date_block_num','shop_id']).agg({'revenue': ['sum']})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)

full_data = pd.merge(full_data, group, on=['date_block_num','shop_id'], how='left')
full_data['date_shop_revenue'] = full_data['date_shop_revenue'].astype(np.float32)

group = group.groupby(['shop_id']).agg({'date_shop_revenue': ['mean']})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)

full_data = pd.merge(full_data, group, on=['shop_id'], how='left')
full_data['shop_avg_revenue'] = full_data['shop_avg_revenue'].astype(np.float32)

full_data['delta_revenue'] = (full_data['date_shop_revenue'] - full_data['shop_avg_revenue']) / full_data['shop_avg_revenue']
full_data['delta_revenue'] = full_data['delta_revenue'].astype(np.float16)

full_data = lag_feature(full_data, [1], 'delta_revenue')

full_data.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'], axis=1, inplace=True)

In [ ]:
full_data['month'] = full_data['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
full_data['days'] = full_data['month'].map(days).astype(np.int8)

In [ ]:
full_data = full_data[full_data.date_block_num > 11]

In [ ]:
%%time
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

full_data = fill_na(full_data)

In [ ]:
full_data.drop(["ID"], axis=1, inplace=True)

In [ ]:
cat_col = ["date_block_num", "shop_id", "item_id", "item_category_id", "month", "days"]
full_data[cat_col] = full_data[cat_col].astype(np.int32)

In [ ]:
X_train = full_data[full_data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
y_train = full_data[full_data.date_block_num < 33]['item_cnt_month']
X_valid = full_data[full_data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
y_valid = full_data[full_data.date_block_num == 33]['item_cnt_month']
X_test = full_data[full_data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
X_test.describe().T

In [ ]:
X_train.drop(["delta_price_lag_1", "delta_price_lag_2", "delta_price_lag_3", "delta_price_lag_4", "delta_price_lag_5", "delta_price_lag_6"], axis=1, inplace=True)
X_valid.drop(["delta_price_lag_1", "delta_price_lag_2", "delta_price_lag_3", "delta_price_lag_4", "delta_price_lag_5", "delta_price_lag_6"], axis=1, inplace=True)
X_test.drop(["delta_price_lag_1", "delta_price_lag_2", "delta_price_lag_3", "delta_price_lag_4", "delta_price_lag_5", "delta_price_lag_6"], axis=1, inplace=True)

In [ ]:
cat_feats = [i for i, c in enumerate(X_train.columns) if c in cat_col]

In [ ]:
del full_data

gc.collect()

In [ ]:
X_train.to_csv("X_train.csv", index=False)
X_valid.to_csv("X_valid.csv", index=False)
X_test.to_csv("X_test.csv", index=False)

y_train.to_csv("y_train.csv", index=False)
y_valid.to_csv("y_valid.csv", index=False)

XGBoost

In [ ]:
%%time
xgb_model = XGBRegressor(max_depth=10,
                         n_estimators=1000, 
                         min_child_weight=1000,
                         colsample_bytree=0.7, 
                         subsample=0.7,
                         eta=0.3, 
                         seed=0)
xgb_model.fit(X_train, 
              y_train, 
              eval_metric="rmse", 
              eval_set=[(X_train, y_train), (X_valid, y_valid)], 
              verbose=20, 
              early_stopping_rounds=10)

In [ ]:
plt.rcParams["figure.figsize"] = (10, 14)
plot_importance(xgb_model)
plt.show()

In [ ]:
%%time

xgb_train_pred = xgb_model.predict(X_train)
xgb_val_pred = xgb_model.predict(X_valid)
xgb_test_pred = xgb_model.predict(X_test)

print('Train rmse:', np.sqrt(mean_squared_error(y_train, xgb_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(y_valid, xgb_val_pred)))

In [ ]:
xgb_model.save_model("xgboost")
del xgb_model, xgb_train_pred
gc.collect()

In [ ]:
submission = pd.DataFrame(test['ID'], columns=['ID'])
submission['item_cnt_month'] = xgb_test_pred.clip(0., 20.)
submission.to_csv('submission_xgb.csv', index=False)
submission.head(5)

LightGBM

In [ ]:
%%time
model=LGBMRegressor(
        n_estimators=200,
        learning_rate=0.03,
        num_leaves=42,
        colsample_bytree=0.95,
        subsample=0.8,
        max_depth=9,
        reg_alpha=0.03,
        reg_lambda=0.07,
        min_split_gain=0.02,
        min_child_weight=40,
        silent=False)


model.fit(X_train, y_train)

In [ ]:
%%time

lgb_train_pred = model.predict(X_train)
lgb_val_pred = model.predict(X_valid)
lgb_test_pred = model.predict(X_test)

print('Train rmse:', np.sqrt(mean_squared_error(y_train, lgb_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(y_valid, lgb_val_pred)))

In [ ]:
model.booster_.save_model('lgb_classifier.txt') 
del model, lgb_train_pred
gc.collect()

In [ ]:
final_test_preds = 0.5 * lgb_test_pred + 0.5 * xgb_test_pred 
final_test_preds = final_test_preds.clip(0., 20.)

In [ ]:
submission = pd.DataFrame(test['ID'], columns=['ID'])
submission['item_cnt_month'] = final_test_preds
submission.to_csv('submission.csv', index=False)
submission.head(5)